In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, SimpleRNN, LSTM, Masking, GRU

Reading Dataset

In [ ]:
ds = pd.read_csv('/content/drive/MyDrive/EFQM/5-Multi label classification/Multivariate_pollution.csv')
ds.head()

Check target values

In [ ]:
print('snow data', ds['snow'].unique())
print('rain data', ds['rain'].unique())

Checking all data

In [ ]:
ds.info()

Creating time features

In [ ]:
ds['date'] = pd.to_datetime(ds['date'])
ds['year'] = ds['date'].dt.year
ds['month'] = ds['date'].dt.month
ds['day'] = ds['date'].dt.day
ds['hour'] = ds['date'].dt.hour


ds.drop(['date'], axis=1, inplace=True)
ds.head()

create dummy data from wind_dir feature

In [ ]:
dummy_wind = pd.get_dummies(ds['wnd_dir'])
data = pd.concat([ds, dummy_wind], axis=1)
ds.drop(['wnd_dir'], axis=1, inplace=True)
ds.head()

Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = ds.drop(columns=['snow', 'rain'])
y = ds[['snow', 'rain']]

scaler = MinMaxScaler()
scaled_x = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

ds = pd.concat([scaled_x, y], axis=1)

ds.head()

Windowing

In [ ]:
X = []
y = []

sequence_length = 16
for i in range(sequence_length, len(ds)):
    X.append(ds.iloc[i-sequence_length:i].values)
    y.append(ds.iloc[i][['snow', 'rain']])

X, y = np.array(X), np.array(y)
X.shape, y.shape

Train, Test, and Validation set splitting

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape

Model

In [ ]:
model = tf.keras.Sequential([
    Masking(mask_value=0., input_shape=(X_train.shape[1], X_train.shape[2])),

    GRU(128, return_sequences=True),
    LeakyReLU(negative_slope=0.01),

    GRU(64),
    LeakyReLU(negative_slope=0.01),

    Dense(32),
    LeakyReLU(negative_slope=0.01),

    Dense(2)
])

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, epochs=32, batch_size=4, validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)])

Plot Training Results

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

Show Test Results

In [ ]:
# Snow prediction
plt.plot(y_pred[:, 0])
plt.plot(y_test[:, 0])
plt.title('Snow prediction')
plt.ylabel('prediction')
plt.xlabel('ith sample')

In [ ]:
# Rain prediction
plt.plot(y_pred[:, 1])
plt.plot(y_test[:, 1])

plt.title('Rain prediction')
plt.ylabel('prediction')
plt.xlabel('ith sample')

Regression Scores

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(X_test)


print('Snow Result')
snow_df = pd.DataFrame(index=['RMSE', 'MSE', 'R2'], columns=['Score'])
snow_df.iloc[0, 0] = np.sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))
snow_df.iloc[1, 0] = mean_squared_error(y_test[:, 0], y_pred[:, 0])
snow_df.iloc[2, 0] = r2_score(y_test[:, 0], y_pred[:, 0])
print(snow_df, '\n')


print('Rain Result')
rain_df = pd.DataFrame(index=['RMSE', 'MSE', 'R2'], columns=['Score'])
rain_df.iloc[0, 0] = np.sqrt(mean_squared_error(y_test[:, 1], y_pred[:, 1]))
rain_df.iloc[1, 0] = mean_squared_error(y_test[:, 1], y_pred[:, 1])
rain_df.iloc[2, 0] = r2_score(y_test[:, 0], y_pred[:, 1])
print(rain_df)